In [1]:
import pandas as pd
import pickle
from pathlib import Path
import os
import pandas as pd
import scanpy as sc
import cellink as cl
from cellink.tl._burden_testing import *
from plotting_report import *

/data/nasif12/home_if12/l_mai/.conda/envs/mm_scgenetics/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Combine all results and do FDR correction

In [2]:
chr1_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr1_all_results.pkl")
chr2_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr2_all_results.pkl")
chr3_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr3_all_results.pkl")
chr4_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr4_all_results.pkl")
chr5_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr5_all_results.pkl")
chr6_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr6_all_results.pkl")
chr7_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr7_all_results.pkl")
chr8_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr8_all_results.pkl")
chr9_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr9_all_results.pkl")
chr10_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr10_all_results.pkl")
chr11_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr11_all_results.pkl")
chr12_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr12_all_results.pkl")
chr13_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr13_all_results.pkl")
chr14_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr14_all_results.pkl")
chr15_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr15_all_results.pkl")
chr16_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr16_all_results.pkl")
chr17_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr17_all_results.pkl")
chr18_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr18_all_results.pkl")
chr19_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr19_all_results.pkl")
chr20_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr20_all_results.pkl")
chr21_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr21_all_results.pkl")
chr22_all_results=pd.read_pickle("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/chr22_all_results.pkl")


In [3]:
all_results = pd.concat([chr1_all_results, 
                    chr2_all_results,
                   chr3_all_results,
                   chr4_all_results,
                   chr5_all_results,
                    chr6_all_results, 
                    chr7_all_results,
                   chr8_all_results,
                   chr9_all_results,
                   chr10_all_results,
                    chr11_all_results,
                   chr12_all_results,
                   chr13_all_results,
                    chr14_all_results, 
                    chr15_all_results,
                   chr16_all_results,
                   chr17_all_results,
                   chr18_all_results,
                   chr19_all_results,
                   chr20_all_results,
                   chr21_all_results,
                   chr22_all_results,], ignore_index=True)

In [17]:
all_results['burden_type'] = all_results['burden_type'].replace('DNA_LM_mixed', 'DNA_LM_combined')
# drop GENE_TSS_DISTANCE as not analysed in this study
all_results = all_results[all_results['burden_type'] != 'GENE_TSS_DISTANCE']
# drop non-immune cell types
all_results[~all_results["cell_type"].isin(["Erythrocytes", "Platelets"])]
all_results.head()

burden_gene      target_gene      burden_type    pvalue      beta  \
0  ENSG00000228463  ENSG00000228463       CADD_PHRED  0.393581 -0.004054   
1  ENSG00000228463  ENSG00000228463        DNA_LM_up  0.104385 -0.007605   
2  ENSG00000228463  ENSG00000228463      DNA_LM_down  0.125772 -0.007172   
3  ENSG00000228463  ENSG00000228463    MAF_beta_1.25  0.339791 -0.004519   
4  ENSG00000228463  ENSG00000228463  DNA_LM_combined  0.104385 -0.007605   

  cell_type chromosome  
0    CD8 ET          1  
1    CD8 ET          1  
2    CD8 ET          1  
3    CD8 ET          1  
4    CD8 ET          1

In [18]:
all_results.to_csv('/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/all_results.csv', index=False) 

### Correcting p-values

In [19]:
df_final, df_to_plot = FDR_correction(all_results)

In [22]:
df_final.head()

burden_gene      target_gene burden_type    pvalue      beta  \
0   ENSG00000228463  ENSG00000228463  CADD_PHRED  0.393581 -0.004054   
7   ENSG00000228327  ENSG00000228327  CADD_PHRED  0.348254 -0.002507   
14  ENSG00000237491  ENSG00000237491  CADD_PHRED  0.617412 -0.001265   
21  ENSG00000225880  ENSG00000225880  CADD_PHRED  0.924011 -0.000300   
28  ENSG00000230368  ENSG00000230368  CADD_PHRED  0.532657  0.002424   

   cell_type chromosome  significant  FDR_corrected celltype-annotation  
0     CD8 ET          1        False       0.955526   CD8 ET-CADD_PHRED  
7     CD8 ET          1        False       0.947938   CD8 ET-CADD_PHRED  
14    CD8 ET          1        False       0.979683   CD8 ET-CADD_PHRED  
21    CD8 ET          1        False       0.997450   CD8 ET-CADD_PHRED  
28    CD8 ET          1        False       0.970296   CD8 ET-CADD_PHRED

In [23]:
df_to_plot.head()

celltype       annotation   n     celltype_annotation
0   CD8 ET       CADD_PHRED  27       CD8 ET_CADD_PHRED
1   CD8 ET        DNA_LM_up   1        CD8 ET_DNA_LM_up
2   CD8 ET      DNA_LM_down   1      CD8 ET_DNA_LM_down
3   CD8 ET    MAF_beta_1.25  19    CD8 ET_MAF_beta_1.25
4   CD8 ET  DNA_LM_combined   7  CD8 ET_DNA_LM_combined

In [20]:
df_to_plot.to_csv('/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/all_results_summary.csv', index=False) 

In [21]:
df_final.to_csv('/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/association/all_results_FDR_corrected.csv', index=False) 

### Combine all burden results

In [25]:
df_burdens = pd.read_parquet("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/burdens/chr1_all_burdens.parquet")
df_burdens["Chromosome"] = ["1" for j in range(len(df_burdens.index))]
for chrom in range(2, 23):
    df_add = pd.read_parquet(f"/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/burdens/chr{chrom}_all_burdens.parquet")
    df_add["Chromosome"] = [str(chrom) for j in range(len(df_add.index))]
    df_burdens = pd.concat([df_burdens, df_add])

df_burdens = df_burdens.drop(columns=['GENE_TSS_DISTANCE'], errors='ignore')

# Rename 'DNA_LM_mixed' to 'DNA_LM_combined'
df_burdens = df_burdens.rename(columns={'DNA_LM_mixed': 'DNA_LM_combined'})


In [26]:
df_burdens.head()

CADD_PHRED  DNA_LM_up  DNA_LM_down  MAF_beta_1.25  DNA_LM_combined  \
id                                                                        
1_1     480.430   0.175453     0.318940    1707.403896         0.318940   
2_2     437.833   0.149238     0.269003    1680.997840         0.269003   
3_3     426.746   0.149238     0.269003    1609.707778         0.269003   
4_4     434.984   0.149238     0.269003    1638.467622         0.269003   
6_6     428.957   0.149238     0.269003    1637.196702         0.269003   

     GENE_TSS_DISTANCE_SAIGE           Geneid Chromosome  
id                                                        
1_1                 0.019757  ENSG00000243485          1  
2_2                 0.020341  ENSG00000243485          1  
3_3                 0.019019  ENSG00000243485          1  
4_4                 0.019117  ENSG00000243485          1  
6_6                 0.019339  ENSG00000243485          1

In [28]:
df_burdens.to_parquet("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/output/burdens/all_burdens.parquet")